In [1]:
# import gc
# import torch
# gc.collect()
# torch.cuda.empty_cache()

In [1]:
!pip install -q numpy==1.26.4
# !pip install -q datasets huggingface-hub
# !pip install -q llama-index llama-index-core llama-index-llms-openai openai llama-index-postprocessor-longllmlingua
!pip install -q accelerate==0.29.2
!pip install -q datasets==2.18.0
!pip install -q llama-index==0.10.7
!pip install -q llama-index-agent-openai==0.1.7
!pip install -q llama-index-core==0.10.28
!pip install -q llama-index-embeddings-openai==0.1.7
!pip install -q llama-index-legacy==0.9.48
!pip install -q llama-index-llms-openai==0.1.15
!pip install -q llama-index-multi-modal-llms-openai==0.1.5
!pip install -q llama-index-postprocessor-longllmlingua==0.1.2
!pip install -q llama-index-program-openai==0.1.5
!pip install -q llama-index-question-gen-openai==0.1.3
!pip install -q llama-index-readers-file==0.1.16
!pip install -q llamaindex-py-client==0.1.18
!pip install -q llmlingua==0.2.2
!pip uninstall transformers -y
!pip install transformers==4.36.2



[notice] A new release of pip is available: 23.3.1 -> 25.0.1
[notice] To update, run: python -m pip install --upgrade pip

[notice] A new release of pip is available: 23.3.1 -> 25.0.1
[notice] To update, run: python -m pip install --upgrade pip

[notice] A new release of pip is available: 23.3.1 -> 25.0.1
[notice] To update, run: python -m pip install --upgrade pip

[notice] A new release of pip is available: 23.3.1 -> 25.0.1
[notice] To update, run: python -m pip install --upgrade pip

[notice] A new release of pip is available: 23.3.1 -> 25.0.1
[notice] To update, run: python -m pip install --upgrade pip
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llama-index-llms-openai 0.1.31 requires llama-index-core<0.11.0,>=0.10.57, but you have llama-index-core 0.10.28 which is incompatible.
llama-index-program-openai 0.1.7 requires llama-index-core<0.11.0,>=0.10.5

In [2]:
from datasets import load_dataset
import pandas as pd
import random
import time
import pickle
import os
import torch
import gc
import openai
from llama_index.core import Document
from llama_index.core import VectorStoreIndex
from llama_index.llms.openai import OpenAI
from llama_index.core.indices.query.schema import QueryBundle
from llama_index.core.response_synthesizers import CompactAndRefine
from llama_index.postprocessor.longllmlingua import LongLLMLinguaPostprocessor

In [4]:
# 비압축 RAG에서 사용할 프롬프트 모델 유형을 설정합니다.
# (압축 RAG의 프롬프트 모델은 기본값으로 "gpt-4o-mini"로 설정되어 있으며,
# 압축 모델은 기본값으로 "NousResearch/Llama-2-7b-hf"로 설정됩니다.)
gpt_model = "gpt-4o-mini"

# 실행할 실험 횟수입니다. 각 실험은 사용자가 특정 Arxiv 출판물에 대해 질문을 하는 RAG 작업을 나타냅니다.
num_of_iterations = 5

# 각 실험에서 Arxiv 출판물은 랜덤하게 선택되므로, 원래 결과의 재현성을 보장하기 위해 고정된 시드를 설정합니다.
set_seed = 0

# 각 실험에서 RAG가 검색할 유사한 청크의 개수입니다.
similarity_top_k = 5

# 검색된 문맥을 압축할 때 목표로 하는 토큰 수입니다.
target_token = 2**9

# OpenAI의 LLM 초기화
llm = OpenAI(model=gpt_model)

# LongLLMLinguaPostprocessor 초기화
postprocessor = LongLLMLinguaPostprocessor(
    model_name="NousResearch/Llama-2-7b-hf",  # ✅ LLaMA2 계열로 교체, 안정적
    # model_name = "beomi/open-llama-2-ko-7b", # 한국어 모델도 가능
    target_token=target_token,
    rank_method="longllmlingua",
    additional_compress_kwargs={
        "condition_compare": True,
        "condition_in_question": "after",
        "context_budget": "+100",
        "reorder_context": "sort",
        "dynamic_context_compression_ratio": 0.3,
    },
)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:896: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/583 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/746 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

...

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/200 [00:00<?, ?B/s]

In [5]:
dataset = load_dataset("ccdv/arxiv-classification")
dataset_train = dataset["train"]
dataset_train_df = pd.DataFrame(dataset_train)

# AI 관련 논문만 유지 (라벨이 #2로 분류된 것만 필터링):
dataset_train_df_ai = dataset_train_df[dataset_train_df["label"] == 2].reset_index(drop=True)

articles = [Document(text=content) for content in dataset_train_df_ai["text"]]

Generating train split:   0%|          | 0/28388 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2500 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2500 [00:00<?, ? examples/s]

In [6]:
query = "Does this publication involve Reinforcement Learning? Answer in a single word, either Yes or No"
# query = "이 출판물이 강화 학습(Reinforcement Learning)과 관련이 있나요? 단어 하나로 대답하세요, Yes 또는 No 중 하나로."

# 결과를 기록할 딕셔너리 설정:
# 두 가지 분류 RAG 간의 일치 여부와 추적 중인 메트릭을 저장:
record = {
    "Yes,Yes": 0,  # 두 방식 모두 긍정
    "No,No": 0,    # 두 방식 모두 부정
    "Yes,No": 0,   # 비압축 방식 긍정, 압축 방식 부정
    "No,Yes": 0,   # 비압축 방식 부정, 압축 방식 긍정
    "original_tokens": [],  # 원래 문맥의 토큰 수
    "compressed_tokens": [],  # 압축된 문맥의 토큰 수
    "ratios": [],  # 압축 비율
    "time_nc": [],  # 비압축 방식의 실행 시간
    "time_c": [],   # 압축 방식의 실행 시간
}

# GPU의 불필요한 캐시 공간을 정리하는 것은 좋은 습관입니다:
gc.collect()
torch.cuda.empty_cache()

# 모든 문서를 반복하지 않기 위해 랜덤하게 문서를 선택합니다:
random.seed(set_seed)
random_iterations = random.sample(range(len(articles)), num_of_iterations)

index = VectorStoreIndex.from_documents(articles[1:2])

# 벡터 DB에서 top_k 청크 검색:
retriever = index.as_retriever(similarity_top_k=similarity_top_k)
retrieved_context = retriever.retrieve(query)
context_list = [chunk.get_content() for chunk in retrieved_context]

# 압축된 문맥으로 LLM에 프롬프트 전달:
new_retrieved_context = postprocessor.postprocess_nodes(
    retrieved_context,
    query_bundle=QueryBundle(query_str=query)
)

In [9]:
# 압축 전 원본 문맥 출력
original_contexts = "\n\n".join([chunk.get_content() for chunk in retrieved_context])
original_tokens = postprocessor._llm_lingua.get_token_length(original_contexts)

print("="*80)
print("📌 [압축 전 문맥]")
print(original_contexts[:3000])  # 너무 길면 앞 부분만 미리 확인
print(f"\n🔢 토큰 수 (압축 전): {original_tokens}")
print("="*80)

# 압축 후 문맥 출력
compressed_contexts = "\n\n".join([chunk.get_content() for chunk in new_retrieved_context])
compressed_tokens = postprocessor._llm_lingua.get_token_length(compressed_contexts)

print("\n" + "="*80)
print("📌 [압축 후 문맥]")
print(compressed_contexts[:3000])  # 마찬가지로 앞부분만 출력
print(f"\n🔢 토큰 수 (압축 후): {compressed_tokens}")
print("="*80)


📌 [압축 전 문맥]
Advances in Neural Information Processing Systems 30, NIPS, 2017. 3
[22] K. Fragkiadaki, P. Agrawal, S. Levine, and J. Malik. Learning visual predictive models of physics for playing billiards.
ICLR, 2016. 4, 7
[23] H. Gao, J. Mao, J. Zhou, Z. Huang, L. Wang, and W. Xu.
Are you talking to a machine? dataset and methods for multilingual image question. In Advances in Neural Information
Processing Systems, pages 2296–2304, 2015. 3
[24] I. Goodfellow, J. Pouget-Abadie, M. Mirza, B. Xu,
D. Warde-Farley, S. Ozair, A. Courville, and Y. Bengio. Generative adversarial nets. In Advances in neural information
processing systems, pages 2672–2680, 2014. 7
[25] J. B. Hamrick, R. Pascanu, O. Vinyals, A. Ballard, N. Heess,
and P. Battaglia. Imagination-based decision making with
physical models in deep neural networks. 4
[26] K. He, X. Zhang, S. Ren, and J. Sun. Deep residual learning for image recognition. In Proceedings of the IEEE Con-

[27]

[28]

[29]

[30]
[31]

[32]

[33]

[34]

[

In [ ]:
# 각 문서에 대해 독립적인 실험을 진행
for document_index in random_iterations:
    # 검색을 위해 관련 문서만 참조하고 이를 벡터 데이터베이스로 처리:
    index = VectorStoreIndex.from_documents(articles[document_index:(document_index + 1)])
    print("\n\n                                                 문서 인덱스:", document_index)
    print("                                                 반복 횟수:", iteration_counter, "/", num_of_iterations)
    iteration_counter += 1

    # RAG 검색: 이 부분은 비압축 및 압축 방식 모두에서 공통:
    retrieval_start_time = time.time()
    # 벡터 DB에서 top_k 청크 검색:
    retriever = index.as_retriever(similarity_top_k=similarity_top_k)
    retrieved_context = retriever.retrieve(query)
    context_list = [chunk.get_content() for chunk in retrieved_context]
    retrieval_time = time.time() - retrieval_start_time

    # 비압축 RAG 방식:
    # 압축 없이 LLM에 프롬프트를 전달하며, 쿼리에 검색된 문맥 청크를 추가:
    start_time_nc = time.time()
    prompt = "\n\n".join(context_list + [query])
    response_nc = llm.complete(prompt)
    time_nc = time.time() - start_time_nc
    print("------------------------------------------------ 비압축 방식 응답: " + str(response_nc))

    original_contexts = "\n\n".join(context_list)
    original_tokens = postprocessor._llm_lingua.get_token_length(original_contexts)

    # 이 시점에서 GPU 메모리를 정리하는 것이 필요합니다:
    del prompt
    del context_list
    del original_contexts
    gc.collect()
    torch.cuda.empty_cache()

    # 압축 문맥 RAG 방식:
    # 압축된 문맥으로 LLM에 프롬프트 전달:
    start_time_c = time.time()
    new_retrieved_context = postprocessor.postprocess_nodes(
        retrieved_context,
        query_bundle=QueryBundle(query_str=query)
    )

    response_c = CompactAndRefine().synthesize(query, new_retrieved_context)

    time_c = time.time() - start_time_c

    print("------------------------------------------------ 압축 방식 응답: " + str(response_c))

    compressed_contexts = "\n\n".join([chunk.get_content() for chunk in new_retrieved_context])

    compressed_tokens = postprocessor._llm_lingua.get_token_length(compressed_contexts)
    ratio = original_tokens / (compressed_tokens + 1)

    print("원래 토큰 수:", original_tokens)
    print("압축된 토큰 수:", compressed_tokens)
    print("압축 비율:", f"{ratio:.2f}배")

    record[str(response_nc).replace(".", "") + "," + str(response_c).replace(".", "")] += 1
    record["original_tokens"].append(original_tokens)
    record["compressed_tokens"].append(compressed_tokens)
    record["ratios"].append(ratio)
    record["time_nc"].append(time_nc + retrieval_time)
    record["time_c"].append(time_c + retrieval_time)

    # GPU의 불필요한 캐시 공간을 정리하는 것은 좋은 습관입니다:
    del compressed_contexts
    del new_retrieved_context
    del retrieved_context
    gc.collect()
    torch.cuda.empty_cache()